In [1]:
#imports libraries
import pandas as pd

In [ ]:
Below are some calls to functions to be tested.

In [ ]:
'''
The following class describes a polygon in a latitudal plane. It has a function to plot the 
plane and a function to sample points within the plane. The sampling method takes number of 
points to be taking from the sample and the number of sample points for parameters.
'''

In [3]:
files = ['SagamoreBeach.csv','Mashpee.csv','sandwich.csv', 
         'lnSandwich.csv', 'bourne.csv','buzzardsBay.csv']

In [ ]:
#open tower files
def csvFilesToDataFrame(files):
    return pd.concat([pd.read_csv(_file) for _file in files])
tower_data = csvFilesToDataFrame(files)
tower_data

In [ ]:
#columns names
col_names = tower_data.columns.get_values()
#Get the columns starting with rep or owner

regex_keys = ['own', 'rep']

#select columns of the data frame starting with given prefixes        
def select_columns(prefix_list, columns):
    return [item for item in col_names 
            if any(item.startswith(prefex) for prefex in prefix_list)]

In [ ]:
#tests for select_columns
owner_rep = select_columns(regex_keys, col_names)
owner = select_columns(['owner'], col_names)
rep = select_columns(['rep'], col_names)

In [ ]:
tower_data.drop_duplicates()

In [ ]:
#select tower owner names
owner_names=tower_data[['owner_entity_name']].drop_duplicates()
#owner_names

In [ ]:
#Validation fields 
exceptions = ['COLONIAL GAS COMPANY']
probably_cell_towers = ['T-Mobile Northeast LLC','']
wireless_or_towers = ['American Towers LLC']

lte = ['Adams Networks','AT&T',('Big River', 'Broad'),
        'BIT Broad',('Bluegrass','Cellular'),('Bug Tussel','Wireless'),'C Spire',
        'ClearTalk','Colorado Valley','ETC',('Evolve', 'Broadband'),
        ('Infrastructure','Networks'),('iWireless'),('Limitless','Mobile'),
       'Mobile Nation',('Mosaic','Telecom'),('Nex-Tech','Wireless'),
        'Nortex','PTCI','PTC',('Redzone','Wireless'),'Rise Broadband',
        'Rock Wireless','Silver Star',('Space Data','Corporation'),
        'Speed Connect','Sprint',('Syringa','Wireless'),'T-Mobile',
        'U.S. Cellular','US Cellular','United Wireless','Verizon',
       ('VTel','Wireless'),'United States Cellular']

In [ ]:
#rtf file processing utility functions

#read from the file
def read_rtf(rtf_file):
    _file = open(rtf_file, 'r')
    rtf_data = _file.read().strip()
    _file.close()
    return rtf_data
import re

#pick the links in the file
def rtf_links(rtf_data):
    return re.findall('\s+(.*?)}}', rtf_data)
#split and clean strings
def split_clean_str(string, split, index):
    return string.split(split)[index].strip()
#clean all the links in the rtf file
def link_text(rtf_links):
    return [split_clean_str(item, '3', 1) for item in rtf_links if not 'http' in item]
#Get the text from all the links in the rtf file
def get_rtf_linkTexts(rtf_file):
    twr_comps = read_rtf(rtf_file)
    twr_comp_links = rtf_links(twr_comps)
    return link_text(twr_comp_links)
    
link_texts = get_rtf_linkTexts('tower_companies.rtf')#link_text(twr_comp_links)

In [ ]:

tower_companies = pd.DataFrame(link_texts, columns=['tower companies'])
tower_companies

In [ ]:
tower_companies

In [ ]:
#Input:
    #name1: Name of a company
    #name2: Name of a company
    #lte_name: Name of a known LTE provider
#output:
    #Checks if name of the lte company is in name1 and name2
    #calculates score based on the how much of the lte provider's name
    #is in both name1 and name2
def commonGrade(name1,name2, lte_name):
    name1 = str(name1).lower()
    name2 = str(name2).lower()
    
    whole = 30.0
    first = 18.0
    dist = 0
    if any([isinstance(nam,float) for nam in [name1, name2, lte_name]]):
            return dist
    if isinstance(lte_name, str):
        lte_name = lte_name.lower()
        
        
        if lte_name in name1 and lte_name in name2:
            return whole;
        return dist
    if isinstance(lte_name, tuple):
        main = lte_name[0].lower()
        extension = lte_name[1].lower()
        if main in name1 and main in name2:
            dist+=first
            if extension in name1 and extension in name2:
                dist+=whole-first
        return dist
    raise ValueError("Found something lte name that's neither string nor tuple!")
            
def both_LTE(name1,name2):
    #print [commonGrade(name1,name2, lte_name) for lte_name in lte]
    return max([commonGrade(name1,name2, lte_name) for lte_name in lte])
'''
Calculates the distance between the names of two companies based on the edit distance and
the number of similar works present
'''
import editdistance as editd
def distance(name1, name2):
    #print "name 1: "+name1
    #print "name 2: "+name2
    _nm1 = str(name1).lower().split(',')[0]#.split(' ')
    _nm2 = str(name2).lower().split(',')[0]#.split(' ')
    nm1 = _nm1.split(' ')
    nm2 = _nm2.split(' ')
   
    bound = min(len(nm1),len(nm2))
    dist = 20.0
    both_lte = both_LTE(name1, name2)
    for i in range(bound):
        if nm1[i]==nm2[i]:
            dist+=dist/((i+1)**2)
        else:
            dist-=dist/((i+1)**2)
    return dist - editd.eval(_nm1, _nm2)+both_lte
                   
    


In [ ]:
#distance('Communication & Control, Inc.','ClearShot Communications')
#lte_name = ('C Spire','Corporation')
name1 = 'C Spire Corporation'
name2 = 'C Spire'
both_LTE(name1,name2)

In [ ]:
#distances array between a names and list of names
def one_to_many(one, many):
    return [(one,item, distance(one, item)) for item in many]
'''
input:
    owners: Owner names of found towers
    towerComp: Known tower companies 
output:
    data frame of (owner, tower companies, names distance)
'''
def owner_to_company_name_dist(owners, towerComp):
    distances = [one_to_many(name[0], towerComp) for name in owners]
    comparision_frame = pd.concat([pd.DataFrame(item, columns = ('tower owner', 'tower company','name distance')) 
                                 for item in distances])
    return comparision_frame
comparision_frame = owner_to_company_name_dist(owner_names.get_values(), tower_companies)

In [ ]:
comparision_frame.to_csv(path_or_buf = 'tower_owner_tower_comp_dist.csv')

In [ ]:
owner_names.get_values()

In [ ]:
#input:
    #html_file: html file
#output: 
    #Tables: list of raw tables extracted from the file
import re
def wekipediaScraper(html_file):
    open_file = open(html_file, 'r')
    file_data = open_file.read()
    open_file.close()
    opening_table = '<table class=[.*?] border=[.*?] cellpadding=[.*?] style=[.*?]>'
    tables= re.findall('(?s)(?<=<table)(.+?)(?=</table>)', file_data)
    #print file_data
    return tables

In [ ]:
#input:
    #tables: list of raw html table contents
#output: 
    #matrix of html table row raw contents
def tableRow(tables):
    return [re.findall('(?s)(?<=<tr)(.+?)(?=</tr>)', table) for table in tables]

In [ ]:
tables = wekipediaScraper('weki.html')
#tables
#tableRow(tables)

In [ ]:
#input: 
    #tr: raw html row content
#output:
    #list of clean text extracted each column of the row
def separateCols(tr):
    cols = re.findall('(?s)(?<=<td)(.+?)(?=/td>)|(?s)(?<=<th)(.+?)(?=/th>)', tr)
    clean_cols = [re.findall('(?s)(?<=>)(.+?)(?=<)',max(col, key = len)) for col in cols]
    cleaned_cols = [max(col,key=len).split('>')[-1] if col else 'NaN' for col in clean_cols]
    
    
    return cleaned_cols
import pandas as pd
#input:
    #table: raw html table content
#output:
    #Data frame of the html table cell text
def table_to_dataframe(table):
    matrix = [separateCols(tr) for tr in table]
    return pd.DataFrame(matrix[1:], columns=matrix[0])
#input:
    #html_file: html file
    #index: index location of a table in the file
#output:
    #dataframe of the cell content of the table with given index
def html_table_to_dataframe(html_file, index):
    tables = wekipediaScraper(html_file)
    row_separated = tableRow(tables)
    return table_to_dataframe(table[index])

In [ ]:
separateCols('<tr><td><a href="/wiki/Big_River_Telephone" title="Big River Telephone">Big River Broadband</a></td><td></td><td><a href="/wiki/3GPP_Long_Term_Evolution" class="mw-redirect" title="3GPP Long Term Evolution">LTE</a></td><td>Unknown</td><td>MO</td><td><a href="/wiki/Big_River_Telephone" title="Big River Telephone">Big River Telephone</a></td></tr>')

In [ ]:
head = '<tr style="background:skyblue;"><th>Operator</th><th>Voice&#160;technology</th><th>Data&#160;technology</th><th data-sort-type="number">Subscribers<br /><small>(in millions)</small></th><th>Coverage<br /><small>(excluding&#160;roaming)</small></th><th>Ownership</th></tr>'
separateCols(head)

In [ ]:
#tables = wekipediaScraper('weki.html')
#tables
table = tableRow(tables)
nation_wide = table_to_dataframe(table[1])
nation_wide

In [ ]:
merges = table_to_dataframe(table[-3])
merges

In [ ]:
operators = nation_wide['Operator'].append(merges['Operator'])

In [ ]:
tel_companies = operators.append(nation_wide['Ownership'])

In [ ]:
all_companies = tel_companies.append(tower_companies['tower companies']).drop_duplicates()
#original_shape = (295,)

In [ ]:
comparison_df = owner_to_company_name_dist(owner_names.get_values(), all_companies)
comparison_df.sort(['name distance'])[21200:]